Let $S(k)$ be the sum of three or more distinct positive integers having the following properties:


No value exceeds k.
The values form a geometric progression.
The sum is maximal.

$S(4) = 4 + 2 + 1 = 7$

$S(10) = 9 + 6 + 4 = 19$

$S(12) = 12 + 6 + 3 = 21$

$S(1000) = 1000 + 900 + 810 + 729 = 3439$

Let $T(n)= \sum_{k=4}^{n}(−1)^{k}S(k)$.

$T(1000) = 2268$

Find $T(10^17)$.

### Facts

$S(k) = n(1 + r + r^2)$

For a geometric series $(nr^2, nr, n) = (a, b, c)$ we have that $b^2 = ac$

### Ideas
##### 3 case 
Find the prime decomposition of $k = p_1^{\gamma_1} \cdot ... \cdot p_l^{\gamma_l}$ and find the smallest square we can take out, let $k'$ be $k$ divided by this smallest factor. This insures that $k \cdot k'$ is still square. This is our first best guess at $c$ (since we know $ac$ is definately square by this procedure). Then find the factor $f = \frac{k}{k'}$, we know that $fk' \leq k$. so we can find the largest square number $\hat{f} \leq f$ and we have that (obviously) $\hat{f}k' \leq k'$ and since $\hat{f}$ is square $\hat{f}k'k$ is also still square and $b = \sqrt{\hat{f}k'k}$

In [139]:
import numpy as np
from __future__ import division

In [140]:
# Helper function to get prime decomposition
def get_primes(n):
    prim_fac = []
    d = 2
    while d*d <= n:
        while (n % d) == 0:
            prim_fac.append(d)  # supposing you want multiple factors repeated
            n //= d
        d += 1
    if n > 1:
       prim_fac.append(n)
    return prim_fac

def prime_counts(n):
    prim_fac = get_primes(n)
    unique = np.unique(prim_fac)
    counts = [prim_fac.count(p) for p in unique]
    return unique, counts

def make_small(counts):
    # we are going to find the smallest power of 2 and subtract it 
    for i in range(len(counts)):
        if counts[i] > 1:
            counts[i] = counts[i] - 2
    return counts
def make_even(counts):
    return [count - (count % 2) for count in counts]

def multiply(prime_nums, counts):
    prod = 1
    for i in range(len(prime_nums)):
        prod *= prime_nums[i] ** counts[i]
    return prod

def smallest_square(n):
    for i in range(int(n)):
        if i**2 >= n:
            return i - 1
    return 1

In [141]:
def s(k):
    # Base case
    if k == 4:
        return 4, 2, 1
    
    # Lower bound from s(k-1)
    l_nrr, l_nr, l_n = s(k-1)
    lower_bound = l_nrr + l_nr + l_n
    
    # Find our first guess of n (the smallest of the series) then improve
    primes, counts = prime_counts(k)
    rep_counts = make_small(counts)
    n = multiply(primes, rep_counts)
    
    factor = k // n
    if factor > 1:
        square_factor = smallest_square(factor) ** 2
        n *= square_factor
        return k, int(np.sqrt(n * k)), n
    else:
        return s(k-1)

In [145]:
s(20)

(20, 10, 5)

In [81]:
9 + 6 + 4

[1]

In [65]:
154350/49

3150.0

In [68]:
prime_counts(154350)

(array([2, 3, 5, 7]), [1, 2, 2, 3])

In [ ]:
154350/